In [1]:
from graphframes import *

ModuleNotFoundError: No module named 'graphframes'

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.types import StringType
from pyspark.sql.types import ArrayType
import re
import time

start_time = time.time()
spark = SparkSession.builder.master("spark://sparkmaster:7077").appName("test").getOrCreate()
sc = spark.sparkContext

/opt/bitnami/spark/python/pyspark/context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning


In [3]:
!pwd

/data/scripts


In [2]:
# read XML, extract title, text and redirect. if not redirect, value is null
initial_df = spark.read.format('xml').options(rowTag='page').load('/data/skwiki-pages-articles.xml')
df = initial_df.selectExpr("title", "revision.text._VALUE as text", "redirect._title as redirect")

In [3]:
# read all titles and replace all _ with spaces
all_titles = spark.read.format("text").load("/data/all_titles_sk.txt")

udf = UserDefinedFunction(lambda x: x.replace("_", " "), StringType())
all_titles = all_titles.select(*[udf(column).alias("value") if column == "value" else column for column in all_titles.columns])
# all_titles.show(11, truncate=False)

In [4]:
# remove all articles that dont have title in all_titles dataset
df = df.join(all_titles, df.title == all_titles.value).select(df["*"])
df.show(5)

+--------------+--------------------+--------------+
|         title|                text|      redirect|
+--------------+--------------------+--------------+
|Hlavná stránka|<templatestyles s...|          null|
|     Main Page|#REDIRECT [[Hlavn...|Hlavná stránka|
|     Esperanto|{{Najlepší článok...|          null|
|    Matematika|{{Substovaný info...|          null|
|  Isaac Newton|{{Infobox Osobnos...|          null|
+--------------+--------------------+--------------+
only showing top 5 rows



In [5]:
wikipedia_namespaces = ["User:", "Wikipedia:", "File:", "MediaWiki:", 
                        "Template:", "Help:", "Category:", "Portal:", 
                        "Draft:", "TimedText:", "Module:", "Gadget:", 
                        "Gadget: definition:", "Special:", "Media:"]

def extract_wikilinks(article):
    all_wikilinks = []
    for wikilink in re.findall("(\[\[.+?\]\])", article):
        wikilink = wikilink.replace("[[", "").replace("]]", "")  # tu bolo lower
        is_namespace = False
        for namespace in wikipedia_namespaces:
            if wikilink.startswith(namespace):
                is_namespace = True
                break
        if is_namespace:
            continue
        # https://en.wikipedia.org/wiki/Help:Wikitext#Wikilinks  <-- cleaning because of this
        if "|" in wikilink:
            wikilink = wikilink.split("|")[0]
        if "#" in wikilink:
            wikilink = wikilink.split("#")[0]
        # NOTE: tu si kontroloval ci je wikilink v all_titles. Ale nejde to. Pri vyhladavani koncovom jednocuho pouzi all_titles set a z neho citaj veci ci tam su alebo nie az.
        if wikilink != "": 
            all_wikilinks.append(wikilink)
    return all_wikilinks



udf = UserDefinedFunction(lambda x: extract_wikilinks(x), ArrayType(StringType()))
df = df.select(*[udf(column).alias("text") if column == "text" else column for column in df.columns])
df = df.selectExpr("title", "text as links", "redirect")
# test.show(5)


In [18]:
all_titles[all_titles["value"] == "asdasdd"].count()

0

In [6]:
df.write.mode("overwrite").json("/data/output/main")
stop_time = time.time()
print(stop_time - start_time)

131.79812622070312
